In [112]:
# !pip install catboost

In [113]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os
from sklearn.decomposition import PCA
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from collections import Counter

In [114]:
data = pd.read_csv("Train_dataset.csv",na_values=["?","unknown"])
data = data.drop(['Registration Number','Cuisine','City'], axis = 1)

In [115]:
data.head()

,Annual Turnover,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,Dedicated Parking,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,42000000,Near Business Hub,14-02-2009,84.3,Not Specific,95.8,1,1,1,0,...,10.0,4.0,NaN,NaN,NaN,8.0,8,6,6,6
1,50000000,Near Party Hub,29-09-2008,85.4,Tier A Celebrity,85.0,1,1,1,1,...,9.0,NaN,4.0,NaN,NaN,5.0,7,7,3,8
2,32500000,Near Business Hub,30-07-2011,85.0,Tier A Celebrity,68.2,1,1,1,1,...,8.0,3.0,NaN,NaN,NaN,7.0,10,5,2,8
3,110000000,Near Party Hub,30-11-2008,85.6,Tier A Celebrity,83.6,0,1,1,1,...,9.0,6.0,NaN,NaN,NaN,7.0,7,4,3,5
4,20000000,Near Party Hub,22-02-2010,NaN,Tier A Celebrity,76.8,1,1,1,1,...,6.0,NaN,2.0,NaN,NaN,NaN,6,2,4,6


In [116]:
key = np.where(data.columns)[0].tolist()
values = list(data.columns.values)
dic = dict(zip(key,values))
dic

{0: 'Annual Turnover',
 1: 'Restaurant Location',
 2: 'Opening Day of Restaurant',
 3: 'Facebook Popularity Quotient',
 4: 'Endorsed By',
 5: 'Instagram Popularity Quotient',
 6: 'Fire Audit',
 7: 'Liquor License Obtained',
 8: 'Situated in a Multi Complex',
 9: 'Dedicated Parking',
 10: 'Open Sitting Available',
 11: 'Resturant Tier',
 12: 'Restaurant Type',
 13: 'Restaurant Theme',
 14: 'Restaurant Zomato Rating',
 15: 'Restaurant City Tier',
 16: 'Order Wait Time',
 17: 'Staff Responsivness',
 18: 'Value for Money',
 19: 'Hygiene Rating',
 20: 'Food Rating',
 21: 'Overall Restaurant Rating',
 22: 'Live Music Rating',
 23: 'Comedy Gigs Rating',
 24: 'Value Deals Rating',
 25: 'Live Sports Rating',
 26: 'Ambience',
 27: 'Lively',
 28: 'Service',
 29: 'Comfortablility',
 30: 'Privacy'}

In [117]:
cat_var = np.where(data.dtypes == np.object)[0]
for i in cat_var:
    data[dic[i]] = data[dic[i]].fillna(data[dic[i]].mode()[0])
num_var = np.where(data.dtypes != np.object)[0]
for i in num_var:
    data[dic[i]] = data[dic[i]].astype(np.float)
    data[dic[i]] = data[dic[i]].fillna(data[dic[i]].median())

In [118]:
import datetime
# Convert "Opening Day of Restaurant" to datetime format
data["Opening Day of Restaurant"] = pd.to_datetime(data["Opening Day of Restaurant"], format='%d-%m-%Y')
# Calculate the age of the restaurant
data["Restaurant Age"] = (datetime.datetime.now() - data["Opening Day of Restaurant"]).dt.days / 365.25
# Drop "Opening Day of Restaurant" column
data.drop("Opening Day of Restaurant", axis=1, inplace=True)


In [119]:
for feature in data.columns: # Loop through all columns in the dataframe
    if data[feature].dtype == 'object': # Only apply for columns with categorical strings
        data[feature] = pd.Categorical(data[feature])# Replace strings with an integer


In [120]:
x_train,x_test,y_train,y_test = train_test_split(data.drop('Annual Turnover', axis = 1).values,data['Annual Turnover'].values,test_size = 0.25,random_state = 100)

In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3493 entries, 0 to 3492
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   Annual Turnover                3493 non-null   float64 
 1   Restaurant Location            3493 non-null   category
 2   Facebook Popularity Quotient   3493 non-null   float64 
 3   Endorsed By                    3493 non-null   category
 4   Instagram Popularity Quotient  3493 non-null   float64 
 5   Fire Audit                     3493 non-null   float64 
 6   Liquor License Obtained        3493 non-null   float64 
 7   Situated in a Multi Complex    3493 non-null   float64 
 8   Dedicated Parking              3493 non-null   float64 
 9   Open Sitting Available         3493 non-null   float64 
 10  Resturant Tier                 3493 non-null   float64 
 11  Restaurant Type                3493 non-null   category
 12  Restaurant Theme               349

In [122]:
x_train.shape

(2619, 30)

In [123]:
cat = CatBoostClassifier(iterations=500,cat_features=cat_var)
model = cat.fit(x_train,y_train)

CatBoostError: ignored

In [41]:
y_pred = model.predict(x_train)
print(classification_report(y_train,y_pred))
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

   3500000.0       1.00      1.00      1.00         2
   4000000.0       1.00      1.00      1.00         2
   4500000.0       1.00      1.00      1.00         2
   5000000.0       1.00      1.00      1.00         4
   5500000.0       1.00      1.00      1.00         1
   6000000.0       1.00      0.89      0.94         9
   6500000.0       1.00      1.00      1.00         2
   7000000.0       1.00      1.00      1.00         4
   7500000.0       1.00      1.00      1.00         5
   8000000.0       1.00      1.00      1.00         8
   8500000.0       1.00      0.90      0.95        10
   9000000.0       1.00      1.00      1.00         8
   9500000.0       1.00      0.91      0.95        23
  10000000.0       0.91      0.85      0.88        75
  10500000.0       1.00      0.88      0.93        32
  11000000.0       0.96      0.85      0.90        27
  11500000.0       1.00      0.89      0.94         9
  12000000.0       0.78    

In [42]:
data_test = pd.read_csv("Test_dataset.csv",na_values=["?","unknown"])
data_test = data_test.drop(['Registration Number'], axis = 1)

In [43]:
data_test.head()

,Cuisine,City,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endoresed By,Instagram Popularity Quotient,Fire Audit,Liquor License Obtained,Situated in a Multi Complex,...,Overall Restaurant Rating,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Live Sports Rating,Ambience,Lively,Service,Comfortablility,Privacy
0,"tibetan,italian",Bangalore,Near Business Hub,13-07-2010,78.00,Tier A Celebrity,69.00,1,1,1,...,6.0,NaN,2.0,NaN,NaN,5,2,2,6,0
1,"tibetan,italian",Hyderabad,Near Party Hub,05-09-2011,89.17,Not Specific,96.00,1,1,1,...,9.0,3.0,NaN,NaN,6.0,7,8,5,1,8
2,"algerian,belgian",Hyderabad,Near Party Hub,12-04-2011,84.00,Not Specific,86.00,1,1,1,...,6.0,3.0,NaN,NaN,NaN,9,7,5,1,5
3,"tibetan,greek",-1,Near Party Hub,16-01-2005,79.38,Not Specific,74.40,0,1,1,...,9.0,6.0,NaN,NaN,NaN,6,7,6,4,7
4,"cuban,british",Pune,Near Party Hub,10-11-2008,84.67,Not Specific,86.46,1,1,1,...,NaN,NaN,NaN,NaN,NaN,4,4,7,7,3


In [44]:
key = np.where(data_test.columns)[0].tolist()
values = list(data_test.columns.values)
dic = dict(zip(key,values))
dic

{0: 'Cuisine',
 1: 'City',
 2: 'Restaurant Location',
 3: 'Opening Day of Restaurant',
 4: 'Facebook Popularity Quotient',
 5: 'Endoresed By',
 6: 'Instagram Popularity Quotient',
 7: 'Fire Audit',
 8: 'Liquor License Obtained',
 9: 'Situated in a Multi Complex',
 10: 'Dedicated Parking',
 11: 'Open Sitting Available',
 12: 'Resturant Tier',
 13: 'Restaurant Type',
 14: 'Restaurant Theme',
 15: 'Restaurant Zomato Rating',
 16: 'Restaurant City Tier',
 17: 'Order Wait Time',
 18: 'Staff Responsivness',
 19: 'Value for Money',
 20: 'Hygiene Rating',
 21: 'Food Rating',
 22: 'Overall Restaurant Rating',
 23: 'Live Music Rating',
 24: 'Comedy Gigs Rating',
 25: 'Value Deals Rating',
 26: 'Live Sports Rating',
 27: 'Ambience',
 28: 'Lively',
 29: 'Service',
 30: 'Comfortablility',
 31: 'Privacy'}

In [45]:
cat_var = np.where(data_test.dtypes == np.object)[0]
for i in cat_var:
    data_test[dic[i]] = data_test[dic[i]].fillna(data_test[dic[i]].mode()[0])
num_var = np.where(data_test.dtypes != np.object)[0]
for i in num_var:
    data_test[dic[i]] = data_test[dic[i]].astype(np.float)
    data_test[dic[i]] = data_test[dic[i]].fillna(data_test[dic[i]].median())

In [47]:
Categorical_columns=["Cuisine","City","Restaurant Location","Endoresed By","Restaurant Type","Restaurant Theme"]
data_test = pd.get_dummies(data_test, columns=Categorical_columns,drop_first=True)

In [48]:
import datetime
# Convert "Opening Day of Restaurant" to datetime format
data_test["Opening Day of Restaurant"] = pd.to_datetime(data_test["Opening Day of Restaurant"], format='%d-%m-%Y')
# Calculate the age of the restaurant
data_test["Restaurant Age"] = (datetime.datetime.now() - data_test["Opening Day of Restaurant"]).dt.days / 365.25
# Drop "Opening Day of Restaurant" column
data_test.drop("Opening Day of Restaurant", axis=1, inplace=True)


In [49]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 164 entries, Facebook Popularity Quotient to Restaurant Age
dtypes: float64(26), uint8(138)
memory usage: 169.1 KB


----

In [59]:
data_test.shape

(500, 164)

In [50]:
y_pred = model.predict(data_test.values)
y_pred

CatBoostError: ignored

In [ ]:
sample = pd.read_csv("Sample Submission.csv")

In [ ]:
final_result = pd.DataFrame({'ID':sample.ID.values,"Term":y_pred})
final_result

In [ ]:
final_result.to_csv('Final_Submission',index=False)